In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
curdir = Path('./statoutput')

# Dati relativi alla validazione sul TEST SET
test_validation_data = pd.DataFrame()
for f in curdir.glob('eval*'):
    filename = f.name
    df = pd.read_pickle(f)
    actfunc = df.get('params.action_activation', ['sigmoid'])
    df['category'] = f"B{df.at[0, 'params.batch']}-E{df.at[0, 'params.epochs']}-H{df.at[0, 'params.hidden_output_dim']}\
-LR{df.at[0, 'params.learning_rate']}-eps{df.at[0, 'params.tolerance']}-ACT-{actfunc[0]}"
    df['filename'] = filename
    test_validation_data = pd.concat([test_validation_data, df])
print(test_validation_data.head())
print(test_validation_data.shape)

   action_accuracy  action_perplexity  attribute_accuracy  \
0         0.840467           4.647734            0.760981   
0         0.844543           4.939461            0.763818   
0         0.851399           3.448803            0.776654   
0         0.845655           4.842728            0.771190   
0         0.841949           4.911037            0.757051   

                                    confusion_matrix  params.batch  \
0  [[751.0, 53.0, 9.0, 7.0, 18.0], [23.0, 1084.0,...            12   
0  [[749.0, 46.0, 12.0, 10.0, 13.0], [26.0, 1081....            12   
0  [[746.0, 49.0, 10.0, 4.0, 11.0], [27.0, 1084.0...            12   
0  [[742.0, 45.0, 13.0, 13.0, 15.0], [29.0, 1086....            12   
0  [[750.0, 45.0, 16.0, 11.0, 12.0], [21.0, 1089....            12   

   params.epochs  params.hidden_output_dim  params.seed  params.learning_rate  \
0             10                       256      9555209               0.00005   
0             10                       256       9

In [2]:
# Dati relativi al training
training_stat = pd.DataFrame()
for f in curdir.glob('stats*'):
    filename = f.name
    df = pd.read_pickle(f)
    df['filename'] = filename
    training_stat = pd.concat([training_stat, df])
print(training_stat.head())
print(training_stat.shape)

       Training Loss  Valid. Loss  Valid. Accur. class.  \
epoch                                                     
1           1.133720     1.064887              0.845147   
2           1.070569     1.071257              0.840307   
3           1.054355     1.063280              0.845147   
4           1.045294     1.057506              0.849986   
5           1.038272     1.064757              0.841731   

       Valid. Accur. mult.label Training Time Validation Time  \
epoch                                                           
1                      0.895531       0:08:33         0:00:28   
2                      0.906633       0:08:37         0:00:27   
3                      0.920011       0:08:34         0:00:28   
4                      0.921719       0:08:34         0:00:27   
5                      0.932252       0:08:34         0:00:27   

                                                 metrics  \
epoch                                                      
1      {'a

In [3]:
# Dati relativi alla validazione durante il training
validation_data = pd.DataFrame()
for f in curdir.glob('testdata*'):
    filename = f.name
    df = pd.read_pickle(f)
    df['filename'] = filename
    validation_data = pd.concat([validation_data, df])
print(validation_data.head())
print(validation_data.shape)

   epoch  batchnum                                     actions_logits  \
0      1         1  [[0.0019575807, 0.999814, 0.032683127, 0.00012...   
1      1         2  [[0.0053882618, 0.99983025, 0.0013555333, 7.51...   
2      1         3  [[0.00012037317, 0.104540944, 0.00040491569, 4...   
3      1         4  [[0.99979585, 0.0031116211, 0.0006431524, 0.00...   
4      1         5  [[0.000258303, 0.0029567834, 0.00024493987, 8....   

                         actions_labels  \
0  [1, 4, 2, 4, 0, 1, 1, 4, 4, 0, 1, 2]   
1  [1, 4, 2, 2, 0, 1, 2, 1, 1, 1, 1, 4]   
2  [4, 1, 1, 1, 4, 2, 1, 2, 0, 1, 1, 1]   
3  [0, 0, 1, 2, 0, 1, 4, 4, 0, 1, 1, 4]   
4  [1, 4, 2, 4, 0, 3, 1, 1, 2, 4, 1, 1]   

                                   attributes_logits  \
0  [[0.00076702854, 0.00038751052, 0.002548172, 0...   
1  [[0.00046720172, 0.00035514444, 0.002924776, 0...   
2  [[0.00017693061, 8.77947e-05, 0.66826177, 0.00...   
3  [[0.00068153336, 0.00036622016, 0.0043590157, ...   
4  [[0.0002490209, 0.0

In [4]:
# Analisi dati per selezione da test dei casi scelti per recupero dati migliore esecuzione
case_study = test_validation_data[test_validation_data['category']=='B12-E6-H768-LR5e-05-eps1e-08-ACT-softmax']
print(case_study.shape)
print(f"MAX action accuracy: {max(case_study['action_accuracy'])}, MAX attribute accuracy: {max(case_study['attribute_accuracy'])}")
print(case_study)
print('Äction accuracy order')
print(case_study.sort_values(by=['action_accuracy']))
print('Ättribute accuracy order')
print(case_study.sort_values(by=['attribute_accuracy']))
print('Order by act_acc+att_acc-act_per')
arbitrary_choice = case_study['action_accuracy'] + case_study['attribute_accuracy'] - case_study['action_perplexity']
case_study['arbitrary_choice'] = arbitrary_choice
print(case_study.sort_values(by=['arbitrary_choice']))

(20, 13)
MAX action accuracy: 0.8525106540670743, MAX attribute accuracy: 0.7966960620999017
   action_accuracy  action_perplexity  attribute_accuracy  \
0         0.852511           5.724079            0.793967   
0         0.845840           6.961841            0.788030   
0         0.850287           5.733896            0.791629   
0         0.847878           6.672284            0.783820   
0         0.847878           6.382444            0.784706   
0         0.846211           6.009374            0.783062   
0         0.849175           6.044589            0.783724   
0         0.852325           5.658315            0.794652   
0         0.848805           6.511304            0.785019   
0         0.851399           6.301615            0.796696   
0         0.850287           6.505223            0.791907   
0         0.850843           6.428665            0.783429   
0         0.847693           6.400397            0.780866   
0         0.849731           6.464630            0.78

C:\Users\marco\AppData\Local\Temp/ipykernel_24644/2666177340.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case_study['arbitrary_choice'] = arbitrary_choice


In [5]:
# Scelta in base a calcolo arbitrario
choice = max(case_study['arbitrary_choice'])
df_test_validation = case_study[case_study['arbitrary_choice']==choice]
print(df_test_validation)
filename = df_test_validation['filename'][0]
training_filename = f"stats{filename[4:]}"
validation_filename = f"testdata{filename[4:]}"
print(f"{training_filename} {validation_filename}")
df_training = training_stat[training_stat['filename']==training_filename]
df_validation = validation_data[validation_data['filename']==validation_filename]
print(f"Dimensioni statistiche training: {df_training.shape}")
print(f"Dimensioni statische validazione training: {df_validation.shape}")

   action_accuracy  action_perplexity  attribute_accuracy  \
0         0.852325           5.658315            0.794652   

                                    confusion_matrix  params.batch  \
0  [[745.0, 44.0, 9.0, 4.0, 11.0], [22.0, 1036.0,...            12   

   params.epochs  params.hidden_output_dim  params.seed  params.learning_rate  \
0              6                       768      4837032               0.00005   

   params.tolerance                                  category  \
0      1.000000e-08  B12-E6-H768-LR5e-05-eps1e-08-ACT-softmax   

               filename params.action_activation  arbitrary_choice  
0  eval-20210820-210814                  softmax         -4.011337  
stats-20210820-210814 testdata-20210820-210814
Dimensioni statistiche training: (6, 8)
Dimensioni statische validazione training: (1758, 9)


In [6]:
print(df_training)

       Training Loss  Valid. Loss  Valid. Accur. class.  \
epoch                                                     
1           1.131759     1.087825              0.830344   
2           1.077110     1.070107              0.845147   
3           1.062885     1.056900              0.857387   
4           1.052512     1.057427              0.855394   
5           1.041906     1.053976              0.857956   
6           1.033517     1.052633              0.859095   

       Valid. Accur. mult.label Training Time Validation Time  \
epoch                                                           
1                      0.900939       0:08:24         0:00:27   
2                      0.920011       0:08:26         0:00:27   
3                      0.927412       0:08:25         0:00:27   
4                      0.936806       0:08:25         0:00:27   
5                      0.946484       0:08:25         0:00:27   
6                      0.946769       0:08:25         0:00:27   

      

In [7]:
print(df_validation)

      epoch  batchnum                                     actions_logits  \
0         1         1  [[3.6151707e-06, 0.99996305, 2.5579902e-05, 1....   
1         1         2  [[8.903495e-06, 0.9998111, 0.000176957, 1.3852...   
2         1         3  [[3.301256e-05, 8.08185e-05, 0.0024472852, 2.8...   
3         1         4  [[0.99978036, 7.0024194e-05, 0.00013504275, 1....   
4         1         5  [[1.9994982e-06, 7.10083e-05, 7.1096147e-06, 2...   
...     ...       ...                                                ...   
1753      6       289  [[8.0385354e-07, 0.9999896, 3.4662537e-06, 3.1...   
1754      6       290  [[1.1502718e-06, 1.1301368e-05, 0.99998677, 1....   
1755      6       291  [[7.399061e-07, 7.820293e-06, 0.99999, 1.54148...   
1756      6       292  [[1.4679453e-07, 9.499526e-07, 8.797774e-07, 4...   
1757      6       293  [[1.3724453e-07, 1.7878556e-06, 0.99999774, 8....   

                            actions_labels  \
0     [1, 4, 2, 4, 0, 1, 1, 4, 4, 0, 1, 2

In [8]:
# Confronto tra tutti i campioni del valore "migliore" con peso perplexity in centesimi
df = test_validation_data.copy()
arbitrary_choice = df['action_accuracy'] + df['attribute_accuracy'] - df['action_perplexity']/100
df['arbitrary_choice'] = arbitrary_choice
print(df.sort_values(by='arbitrary_choice', ascending=False).loc[:, ['action_accuracy', 'action_perplexity', 'attribute_accuracy', 'arbitrary_choice', 'category']])

    action_accuracy  action_perplexity  attribute_accuracy  arbitrary_choice  \
0          0.851399           3.448803            0.776654          1.593565   
0          0.854178           2.596618            0.764734          1.592947   
0          0.855290           2.659773            0.762470          1.591162   
0          0.852325           5.658315            0.794652          1.590395   
0          0.852511           5.724079            0.793967          1.589237   
..              ...                ...                 ...               ...   
0          0.846211           5.516973            0.756700          1.547741   
0          0.844729           4.526007            0.740875          1.540343   
0          0.844729           4.439824            0.730127          1.530457   
0          0.845470           3.485176            0.703981          1.514599   
0          0.848064           3.285109            0.664867          1.480080   

                                     ca

In [9]:
# Confronto tra tutti i campioni del valore "migliore" con peso perplexity in centesimi ma dimezzato
df = test_validation_data.copy()
arbitrary_choice = df['action_accuracy'] + df['attribute_accuracy'] - df['action_perplexity']/200
df['arbitrary_choice'] = arbitrary_choice
print(df.sort_values(by='arbitrary_choice', ascending=False).loc[:, ['action_accuracy', 'action_perplexity', 'attribute_accuracy', 'arbitrary_choice', 'category']])

    action_accuracy  action_perplexity  attribute_accuracy  arbitrary_choice  \
0          0.852325           5.658315            0.794652          1.618686   
0          0.852511           5.724079            0.793967          1.617857   
0          0.851399           6.301615            0.796696          1.616587   
0          0.850287           5.733896            0.791629          1.613247   
0          0.850287           6.132042            0.793463          1.613090   
..              ...                ...                 ...               ...   
0          0.850287           4.057148            0.741263          1.571264   
0          0.844729           4.526007            0.740875          1.562974   
0          0.844729           4.439824            0.730127          1.552657   
0          0.845470           3.485176            0.703981          1.532025   
0          0.848064           3.285109            0.664867          1.496505   

                                    cat

In [10]:
# Confronto tra tutti i campioni del valore "migliore" con peso perplexity in millesimi
df = test_validation_data.copy()
arbitrary_choice = df['action_accuracy'] + df['attribute_accuracy'] - df['action_perplexity']/1000
df['arbitrary_choice'] = arbitrary_choice
print(df.sort_values(by='arbitrary_choice', ascending=False).loc[:, ['action_accuracy', 'action_perplexity', 'attribute_accuracy', 'arbitrary_choice', 'category']])

    action_accuracy  action_perplexity  attribute_accuracy  arbitrary_choice  \
0          0.851399           6.301615            0.796696          1.641793   
0          0.852325           5.658315            0.794652          1.641320   
0          0.852511           5.724079            0.793967          1.640754   
0          0.850287           6.132042            0.793463          1.637618   
0          0.850287           5.733896            0.791629          1.636182   
..              ...                ...                 ...               ...   
0          0.850287           4.057148            0.741263          1.587493   
0          0.844729           4.526007            0.740875          1.581078   
0          0.844729           4.439824            0.730127          1.570416   
0          0.845470           3.485176            0.703981          1.545965   
0          0.848064           3.285109            0.664867          1.509646   

                                    cat

In [12]:
import matplotlib.pyplot as plt
import seaborn as sn
import plotly.figure_factory as ff

znp = np.array(df_test_validation.confusion_matrix[0])
print("original:")
print(znp)
z = znp.transpose()
print("transpose:")
print(z)
x = ['AddToCart', 'None', 'SearchDatabase', 'SearchMemory', 'SpecifyInfo']
y = ['AddToCart', 'None', 'SearchDatabase', 'SearchMemory', 'SpecifyInfo']
     
# change each element of z to type string for annotations
z_text = [[str(y) for y in x] for x in z]
     
# set up figure 
fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Actions confusion matrix</b></i>',title_x=0.5
                  #xaxis = dict(title='x'),
                  #yaxis = dict(title='x')
                 )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted value",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.25,
                        y=0.5,
                        showarrow=False,
                        text="Real value",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(margin=dict(t=50, l=200))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()
# fig.write_image("plots/action_confusion_matrix.png")

original:
[[ 745.   44.    9.    4.   11.]
 [  22. 1036.   37.    8.   29.]
 [  12.  223.  779.   98.    8.]
 [   0.    0.    0.    0.    0.]
 [  14.   83.  119.   76. 2040.]]
transpose:
[[ 745.   22.   12.    0.   14.]
 [  44. 1036.  223.    0.   83.]
 [   9.   37.  779.    0.  119.]
 [   4.    8.   98.    0.   76.]
 [  11.   29.    8.    0. 2040.]]
